# LSTM Training

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
import pandas as pd
import numpy as np
from np import nan
import itertools
import os
import random

ImportError: No module named 'np'

In [87]:
# read finished Data; case training
def readData(src, noConfFlag=True):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
            if noConfFlag:
                dataseries[-1] = np.append(dataseries[-1][:2],np.delete(dataseries[-1][2:], slice(None, None, 3)))
        except ValueError:
            break
        except IndexError:
            if noConfFlag:
                dataseries.append(np.zeros(25*2))
            else:
                dataseries.append(np.zeros(25*3))
    return dataseries,i

In [5]:
labelNames=['rennen','gehen']
dataPaths=['data/output/rennen/','data/output/gehen/']
samplePaths=[]
data=[]
labels=[]
#collect paths to data
dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')

#shuffel data
random.shuffle(samplePaths)
#print(samplePaths)
minLength = 75
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    #if i<minLength:
    #    minLength = i
    data.append(td[10:60])
    if l == '0':
        labels.append(np.array([1,0]))
    elif l == '1':
        labels.append(np.array([0,1]))
    else:
        #labels.append(np.array([0,0]))
        print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

.............................................data loaded.
min Length:  75


In [7]:
#define constants

time_steps=50
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=50
#learning rate for adam
learning_rate=0.001

n_classes=2
#size of batch
batch_size=5

In [8]:
#shape for LSTM
unscaledData = np.concatenate(data, axis=0).reshape((-1,time_steps*50))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [9]:
print(unscaledData.shape,l.shape)
print(unscaledData)
print(l)

(45, 2500) (45, 2)
[[ 16.1076  270.672     3.73877 ... 441.613   266.727   436.42   ]
 [ 25.2767  277.116     3.72409 ... 453.332   211.867   446.808  ]
 [620.342   222.31    632.063   ... 393.256   420.699   399.768  ]
 ...
 [690.804   254.883   706.475   ... 441.553   472.867   438.949  ]
 [  0.        0.      345.047   ... 397.288   441.629   398.571  ]
 [  0.        0.        0.      ... 421.968   590.294   411.513  ]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


### TODO:
- Normalize/Standardize data

In [53]:
#naiv standardize
minval = np.min(unscaledData[np.nonzero(unscaledData)])
maxval = np.max(unscaledData[np.nonzero(unscaledData)])
print(minval,maxval)
d = (unscaledData-minval)/(maxval-minval)
minval = np.min(d[np.nonzero(d)])
maxval = np.max(d[np.nonzero(d)])
print(minval,maxval)

3.08494 727.385
-0.004259201635300154 1.0


In [85]:
n = unscaledData.shape[1]//50
unscaledData[unscaledData == 0] = np.nan
for video in unscaledData:
    for i in range(0,n):
        p=50
        frame = video[p*i:p*(i+1)]
        x_avg = np.nanmean(frame[::2])
        y_avg = np.nanmean(frame[1::2])
        frame[isnan(frame[::2])] = x_avg
        print(frame, x_werte, y_werte)
                
            

NameError: name 'isnan' is not defined

In [10]:
#split data in train and testset
trainSize=30 

train_data = d[:trainSize,:]
train_labels = l[:trainSize,:]
test_data = d[trainSize:,:]
test_labels = l[trainSize:,:]

In [11]:
test_data.shape, test_labels.shape

((15, 2500), (15, 2))

In [12]:
train_data.shape, train_labels.shape

((30, 2500), (30, 2))

In [13]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [14]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [15]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [16]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [17]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [18]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=0
    while iter<trainSize//batch_size:
        print(iter)
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]
        
        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter%5==0 and iter!=0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

0
1
2
3
4
5
For iter  5
Accuracy  0.6
Loss  2.4961517
__________________
Testing Accuracy: 0.6


In [19]:
sess.close()

# Visualisation

In [20]:

show_data = unscaledData.reshape(-1,n_input)
show_label = [np.where(r==1)[0][0] for r in l]

In [21]:
#show_label

In [22]:
show_data.shape

(2250, 50)

In [23]:
testing = show_data[1]
testing[1]

0.0

In [24]:
pairs = [
        (1,8),(1,2),(1,5),(2,3),(3,4),   
        (5,6),(6,7),(8,9),(9,10),(10,11),
        (8,12),(12,13),(13,14),(1,0),(0,15), 
        (15,17),(0,16),(16,18),
        (14,19),(19,20),(14,21),(11,22),(22,23),
        (11,24)
        ]

In [25]:
pairs[0][1]

8

In [26]:
import pygame
import sys, traceback

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((900,750))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        currentSample=int(count/n_input)%len(show_label)
        text = "FPS: {0:.2f}   Playtime: {1:.2f}  NR: {2:d}  Label: ".format(clock.get_fps(), playtime, currentSample+1)
        text += labelNames[show_label[currentSample]]
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        
        for p in pairs:
            x1=int(f[p[0]*2])
            y1=int(f[p[0]*2+1])
            x2=int(f[p[1]*2])
            y2=int(f[p[1]*2+1])
            if not (x1 == 0 or x2 == 0 or y1 == 0 or y2 == 0):
                pygame.draw.line(background, (0, 0, 0), [x1,y1], [x2,y2], 1)
        for i in range(0,f.shape[0],2):#2 for no conf
            #confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            #pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
            if i!=25:
                pygame.draw.rect(background, (0, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
            else:
                pygame.draw.rect(background, (255, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
        screen.blit(background, (300-f[8*2],300-f[8*2+1]))#2 for no conf
        #screen.blit(background, (0,0))
        #Update Pygame display.
        pygame.display.update()

        count += 1
except TypeError:
    print("Type error: ", count," ", i)
    traceback.print_exc(file=sys.stdout)
except:
    traceback.print_exc(file=sys.stdout)
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


error: No available video device

In [29]:
pygame.display.list_modes()

error: video system not initialized

# Live Data-read

In [ ]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [ ]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()